# CS295/395: Secure Distributed Computation
## Homework 7

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request

## Definitions: Paillier Cryptosystem

In [ ]:
import random
import sys
import math

def ipow(a, b, n):
    """calculates (a**b) % n via binary exponentiation, yielding itermediate
       results as Rabin-Miller requires"""
    A = a = a % n
    yield A
    t = 1
    while t <= b:
        t <<= 1

    # t = 2**k, and t > b
    t >>= 2
    
    while t:
        A = (A * A) % n
        if t & b:
            A = (A * a) % n
        yield A
        t >>= 1

def rabin_miller_witness(test, possible):
    """Using Rabin-Miller witness test, will return True if possible is
       definitely not prime (composite), False if it may be prime."""    
    return 1 not in ipow(test, possible-1, possible)

smallprimes = (2,3,5,7,11,13,17,19,23,29,31,37,41,43,
               47,53,59,61,67,71,73,79,83,89,97)

def default_k(bits):
    return int(max(40, 2 * bits))

def is_probably_prime(possible, k=None):
    if possible == 1:
        return True
    if k is None:
        k = default_k(possible.bit_length())
    for i in smallprimes:
        if possible == i:
            return True
        if possible % i == 0:
            return False
    for i in range(k):
        test = random.randrange(2, possible - 1) | 1
        if rabin_miller_witness(test, possible):
            return False
    return True

def generate_prime(bits, k=None):
    """Will generate an integer of b bits that is probably prime 
       (after k trials). Reasonably fast on current hardware for 
       values of up to around 512 bits."""    
    assert bits >= 8

    if k is None:
        k = default_k(bits)

    while True:
        possible = random.randrange(2 ** (bits-1) + 1, 2 ** bits) | 1
        if is_probably_prime(possible, k):
            return possible

In [ ]:
def invmod(a, p, maxiter=1000000):
    """The multiplicitive inverse of a in the integers modulo p:
         a * b == 1 mod p
       Returns b.
       (http://code.activestate.com/recipes/576737-inverse-modulo-p/)"""
    if a == 0:
        raise ValueError('0 has no inverse mod %d' % p)
    r = a
    d = 1
    for i in range(min(p, maxiter)):
        d = ((p // r + 1) * d) % p
        r = (d * a) % p
        if r == 1:
            break
    else:
        raise ValueError('%d has no inverse mod %d' % (a, p))
    return d

def modpow(base, exponent, modulus):
    """Modular exponent:
         c = b ^ e mod m
       Returns c.
       (http://www.programmish.com/?p=34)"""
    result = 1
    while exponent > 0:
        if exponent & 1 == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class PrivateKey(object):

    def __init__(self, p, q, n):
        self.l = (p-1) * (q-1)
        self.m = invmod(self.l, n)

    def __repr__(self):
        return '<PrivateKey: %s %s>' % (self.l, self.m)

class PublicKey(object):

    @classmethod
    def from_n(cls, n):
        return cls(n)

    def __init__(self, n):
        self.n = n
        self.n_sq = n * n
        self.g = n + 1

    def __repr__(self):
        return '<PublicKey: %s>' % self.n

def generate_keypair(bits):
    p = generate_prime(bits / 2)
    q = generate_prime(bits / 2)
    n = p * q
    return PrivateKey(p, q, n), PublicKey(n)

def encrypt(pub, plain):
    while True:
        r = generate_prime(round(math.log(pub.n, 2)))
        if r > 0 and r < pub.n:
            break
    x = pow(r, pub.n, pub.n_sq)
    cipher = (pow(pub.g, plain, pub.n_sq) * x) % pub.n_sq
    return cipher

def decrypt(priv, pub, cipher):
    x = pow(cipher, priv.l, pub.n_sq) - 1
    plain = ((x // pub.n) * priv.m) % pub.n
    return plain

In [ ]:
def e_add(pub, a, b):
    """Add one encrypted integer to another"""
    # YOUR CODE HERE
    raise NotImplementedError()

## Question 1 (30 points)

Implement an electronic voting system using the Paillier cryptosystem. The system has two parts: a voter encrypts their vote using the cryptosystem and submits it to the server, and the server stores the encrypted votes and tallies them at the end of the election. The election server holds the public and private keys; the public key is available to the voters, so that they can encrypt their votes. Each voter should be able to verify that the server has correctly stored their vote, by consulting the encrypted votes to check that their vote indeed appears. Generate your public and private keys with at least 32-bit security.

*Reference:* [Prêt à Voter with Paillier encryption](https://dl.acm.org/doi/10.1016/j.mcm.2008.05.015).

In [ ]:
voting_table = ['Donald Trump',
                'Joseph Biden',
                'Kanye West']

class Voter:
    def vote(self, candidate, server):
        """Submits an encrypted vote to the server"""
        # YOUR CODE HERE
        raise NotImplementedError()

class ElectionServer:
    def __init__(self):
        self.votes = []
        self.sk, self.pk = generate_keypair(32)
    
    def get_public_key(self):
        """Get the public key from the election server"""
        return self.pk
    
    def submit_vote(self, ct_vote_vector):
        """Submit an (encrypted) vote to the election server"""
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def show_votes(self):
        """Show the submitted (encrypted) votes"""
        return self.votes
    
    def tally_votes(self):
        """Tally up the votes at the end of the election. Returns a list of totals in the 
        same order as the list of candidates."""
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE
es = ElectionServer()

Voter().vote('Donald Trump', es)
Voter().vote('Donald Trump', es)
Voter().vote('Joseph Biden', es)
Voter().vote('Joseph Biden', es)

#print(e.show_billboard())
assert es.tally_votes() == [2, 2, 0]

## Question 2 (10 points)

In 2-5 sentences, answer the following:

- What trust assumptions do we make about the *election server* in this election system?
- What trust assumptions do we make about the *voter* in this election system?

YOUR ANSWER HERE

## Question 3 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *election server* could break the rules of the election?
- What are two ways a malicious *voter* could break the rules of the election?

YOUR ANSWER HERE

## Question 4 (30 points)

The [Boston Women's Workforce Council Gender Pay Gap Survey](https://thebwwc.org/mpc) uses MPC to deploy a survey of Boston-area businesses to determine how women and men are paid differently. Each business submits encrypted values for their employees' salaries, and the system calculates the average salaries for women and men across all of the businesses. This design protects the privacy of individual employees, and protects individual businesses from embarrassment.

Implement a system for conducting a survey like this using the Paillier cryptosystem. Participants should submit their own salaries, and specify their gender. The survey server should collect responses, and at the end of the survey, calculate the average salaries for men and women. Use at least 32-bit security.

In [ ]:
class GenderPayGapSurveyParticipant:
    def submit_salary(self, salary, gender, server):
        """Submits an encrypted survey response to the server"""

        # YOUR CODE HERE
        raise NotImplementedError()

class GenderPayGapSurveyServer:
    def __init__(self):
        self.salaries = []
        self.genders = []
        self.sk, self.pk = generate_keypair(32)
    
    def get_public_key(self):
        return self.pk
        
    def submit_salary(self, ct_salary_vector, ct_gender_vector):
        """Store an entry in the survey"""

        # YOUR CODE HERE
        raise NotImplementedError()
    
    def show_salaries(self):
        """Display the (encrypted) submitted salaries"""
        return self.salaries
    
    def compute_average_salaries(self):
        """Tally the results, decrypt, and return a 2-tuple: (average female salary, average male salary)"""
        
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE
s = GenderPayGapSurveyServer()
GenderPayGapSurveyParticipant().submit_salary(10000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(30000, 'Female', s)
GenderPayGapSurveyParticipant().submit_salary(15000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(20000, 'Female', s)
assert s.compute_average_salaries() == (25000.0, 12500.0)

## Question 5 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *survey server* could break the rules of the survey?
- What are two ways a malicious *survey participant* could break the rules of the survey?

YOUR ANSWER HERE